# Volatility Forecasts (Part 2 - XGBoost-STES)

This notebook demonstrates the implementation of the Smooth Transition Exponential Smoothing (STES) model. The model is a variant of the Exponential Smoothing (ES) model that captures non-linear dependencies in volatility time series. The STES model is a more advanced version of the ES model that can capture non-linear dependencies in volatility time series. XGBoost-STES is an extension of STES that uses XGBoost to enhance the STES model by better capturing non-linear dependencies in volatility time series.

This notebook corresponds to the blog series [Volatility Forecasts (Part 2 - XGBoost-STES)](https://steveya.github.io/2024/07/12/volatility-forecast-2.html). We have refactored the code used [Volatility Forecasts (Part 1 - STES Models)](https://steveya.github.io/blob/notebooks/volatility_forecast_1.ipynb) in The aim is to replace the logistic function used by the STES model with a xgboost model, and evaluate their relative performance. It is a work in progress and will be updated as I wrap up my implementations.

In [10]:
%load_ext autoreload
%autoreload 2

import os, sys

project_dir = os.path.abspath('..')
sys.path.insert(0, project_dir)

import pandas as pd
import numpy as np
import pandas_market_calendars as mcal
from typing import List, Any, Optional, Tuple
from enum import Enum
from functools import partial
from collections import defaultdict

import optuna
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from sklearn.metrics import make_scorer
from scipy.stats import randint, uniform, loguniform

from volatility_forecast.data.base import DateLike
from volatility_forecast.data.datamanager import (
    LagReturnDataManager,
    LagAbsReturnDataManager, 
    LagSquareReturnDataManager,
    SquareReturnDataManager,
)
from volatility_forecast.model.stes_model import STESModel
from volatility_forecast.model.xgboost_stes_model import XGBoostSTESModel, DEFAULT_XGBOOST_PARAMS
from volatility_forecast.evaluation.model_evaluator import evaluate_model, compare_models, root_mean_squared_error


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data Provider

In [11]:
ModelName = Enum(
    "ModelName", "ES STES_AE STES_SE STES_ESE STES_EAE STES_AESE STES_EAESE XGBoost_STES"
)

### Equity Data Provider
The function provides the features, labels, and auxiliary equity return data used to fit different models

In [12]:
def equity_data_provider(tickers: Tuple[str], start_date: DateLike, end_date: DateLike, model_name: ModelName) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    returns = LagReturnDataManager().get_data(tickers, start_date, end_date) * 1e2
    realized_var = SquareReturnDataManager().get_data(tickers, start_date, end_date) * 1e4
    feature_sets = np.hstack([
        LagReturnDataManager().get_data(tickers, start_date, end_date) * 1e2,
        LagAbsReturnDataManager().get_data(tickers, start_date, end_date) * 1e2,
        LagSquareReturnDataManager().get_data(tickers, start_date, end_date) * 1e4,
    ])
    if model_name == ModelName.ES:
        return np.ones((len(returns), 1)), realized_var, returns
    elif model_name == ModelName.STES_AE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [1]], ]), realized_var, returns
    elif model_name == ModelName.STES_SE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [2]], ]), realized_var, returns
    elif model_name == ModelName.STES_EAE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [0, 1]], ]), realized_var, returns
    elif model_name == ModelName.STES_ESE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [0, 2]], ]), realized_var, returns
    elif model_name == ModelName.STES_AESE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [1, 2]], ]), realized_var, returns
    elif model_name == ModelName.STES_EAESE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets, ]), realized_var, returns
    elif model_name == ModelName.XGBoost_STES:
        return feature_sets, realized_var, returns
    else:
        raise ValueError(f"Unknown model name: {model_name}")


### Simulated Data Provider
The function provides the features, labels, and auxiliary simulated return data used to fit different models

In [13]:
def simulate_contaminated_garch(n, mu, omega, alpha, beta, eta):
    returns = np.zeros(n)
    sigma2s = np.zeros(n)
    shocks = (np.random.uniform(0, 1, n) < 0.005).astype(float)
    for t in range(1, n):
        sigma2s[t] = omega + alpha * returns[t-1]**2 + beta * sigma2s[t-1]
        returns[t] = np.random.normal(mu, np.sqrt(sigma2s[t])) + eta * shocks[t]

    return returns, sigma2s

def simulated_garch_data_provider(model_name: ModelName) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    returns, latent_var = simulate_contaminated_garch(n=2500, mu=0, omega=0.02, alpha=0.11, beta=0.87, eta=4)
    realized_var = returns[1:] ** 2
    returns = returns[:-1]
    feature_sets = np.vstack([
        returns,
        abs(returns),
        returns**2,
    ]).T
    if model_name == ModelName.ES:
        return np.ones((len(returns), 1)), realized_var, returns
    elif model_name == ModelName.STES_AE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [1]], ]), realized_var, returns
    elif model_name == ModelName.STES_SE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [2]], ]), realized_var, returns
    elif model_name == ModelName.STES_EAE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [0, 1]], ]), realized_var, returns
    elif model_name == ModelName.STES_ESE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [0, 2]], ]), realized_var, returns
    elif model_name == ModelName.STES_AESE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets[:, [1, 2]], ]), realized_var, returns
    elif model_name == ModelName.STES_EAESE:
        return np.hstack([np.ones((len(returns), 1)), feature_sets, ]), realized_var, returns
    elif model_name == ModelName.XGBoost_STES:
        return feature_sets, realized_var, returns
    else:
        raise ValueError(f"Unknown model name: {model_name}")


## Model Fitting and Evaluation

In [14]:
sim_models = {m: STESModel() if m != ModelName.XGBoost_STES else XGBoostSTESModel(**DEFAULT_XGBOOST_PARAMS) for m in ModelName}
spy_models = {m: STESModel() if m != ModelName.XGBoost_STES else XGBoostSTESModel(**DEFAULT_XGBOOST_PARAMS) for m in ModelName}

### Fitting to Simulated Data

In [52]:
num_runs = 1000
simulated_results = defaultdict(float)

np.random.seed(0)
for model_name in ModelName:
    rand_seeds = np.random.randint(0, 1e6, size=num_runs)
    for i in range(num_runs):
        np.random.seed(rand_seeds[i])

        model = sim_models[model_name]
        data_provider = partial(
            simulated_garch_data_provider, 
            model_name=model_name,
        )
        
        os_res, is_res, _, _ = evaluate_model(
            data_provider, 
            model, 
            root_mean_squared_error, 
            500, 2000
        )
        simulated_results[model_name] += res

for model_name in ModelName:
    simulated_results[model_name] /= num_runs

In [29]:
simulated_results

defaultdict(float,
            {<ModelName.ES: 1>: 2.851265469332402,
             <ModelName.STES_AE: 2>: 2.8864909620674513,
             <ModelName.STES_SE: 3>: 2.821076842096175,
             <ModelName.STES_ESE: 4>: 2.938132876527599,
             <ModelName.STES_EAE: 5>: 2.9126491851780054,
             <ModelName.STES_AESE: 6>: 2.9414660242775197,
             <ModelName.STES_EAESE: 7>: 2.8094387526006788,
             <ModelName.XGBoost_STES: 8>: 2.917599319542367})

### Fitting to Equity Return Data

In [4]:
tickers = ("SPY", )
start_date = "2000-01-01"
end_date = "2023-12-31"

In [84]:
spy_results = {}
for model_name in ModelName:
    np.random.seed(0)

    model = spy_models[model_name]
    data_provider = partial(
        equity_data_provider, 
        tickers=tickers, 
        start_date=start_date, 
        end_date=end_date, 
        model_name=model_name
    )

    os_res, is_res, _, _ = evaluate_model(
        data_provider,
        model, 
        root_mean_squared_error,
        10, 4000
    )
    spy_results[model_name] = (os_res, is_res)


In [85]:
spy_results

{<ModelName.ES: 1>: (4.639087790640113, 4.9998967674740395),
 <ModelName.STES_AE: 2>: (4.54443786652832, 4.96971004047666),
 <ModelName.STES_SE: 3>: (4.520988501731453, 4.965343632646092),
 <ModelName.STES_ESE: 4>: (4.497560951862506, 4.932181685635686),
 <ModelName.STES_EAE: 5>: (4.519543044844543, 4.954716010899248),
 <ModelName.STES_AESE: 6>: (4.496005900247276, 4.96213338201455),
 <ModelName.STES_EAESE: 7>: (4.488055353719537, 4.927956913611339),
 <ModelName.XGBoost_STES: 8>: (4.3781566115070865, 5.011798597677629)}

### XGBoost-STES Feature Importance
One added benefit for using the XGBoost in the STES model is the ability to provide feature importance scores. In STES model we can look at the magnitude of the fitted model parameters but since the variables have different magnitude, the interpretation of the fitted parameters can be challenging. 

In [50]:
spy_models[ModelName.XGBoost_STES].model.get_score(importance_type='gain')

{'f0': 3109.922607421875, 'f1': 1948.13232421875, 'f2': 2431.680908203125}

## Hyperparameter Tuning of the XGBoost-STES Model 

The untuned XGBOost-STES model already outperform the other simple STES models. We now test if tuning help with test performance.

In [57]:
spy_data_provider = partial(
    equity_data_provider, 
    tickers=tickers, 
    start_date=start_date, 
    end_date=end_date, 
    model_name=ModelName.XGBoost_STES,
)

### Tuning using Sklearn Randomized Search

In [68]:
import numpy as np
from scipy.stats import uniform, randint
from sklearn.model_selection import ParameterSampler, TimeSeriesSplit

def random_cv_tune_xgboost_model(data_provider):
    param_distributions = {
        'num_boost_round': [1, 5, 10, 20],
        'max_depth': randint(1, 5),
        'learning_rate': loguniform(0.01, 10),
        'reg_lambda': loguniform(0.1, 10.0),
        'colsample_bytree': uniform(0.1, 0.6),
        'colsample_bylevel': uniform(0.1, 0.6),
        'colsample_bynode': uniform(0.1, 0.6),
    }

    # Generate parameter combinations
    param_list = list(ParameterSampler(param_distributions, n_iter=80, random_state=0))

    # Prepare cross-validation
    tscv = TimeSeriesSplit(n_splits=3)

    best_score = float('inf')
    best_params = None

    indices = np.ones((4000, 1))
    # Iterate over parameter combinations
    for params in param_list:
        scores = []

        # Perform cross-validation
        for train_index, val_index in tscv.split(indices):
            # Create and train the model
            model = XGBoostSTESModel(**params)
            rmse, _, _, _ = evaluate_model(
                data_provider, 
                model, 
                root_mean_squared_error, 
                0, 
                val_index[0], 
                val_index[-1]
            )
            scores.append(rmse)

        # Calculate mean RMSE across folds
        mean_rmse = np.mean(scores)

        # Update best score and parameters if better
        if mean_rmse < best_score:
            best_score = mean_rmse
            best_params = params

        # Print progress (optional)
        print(f"Params: {params}")
        print(f"Mean RMSE: {mean_rmse}")
        print("--------------------")

    return best_params, best_score

In [69]:
# Perform hyperparameter tuning
rcv_best_params, rcv_best_score = random_cv_tune_xgboost_model(spy_data_provider)
print("Best Parameters:", rcv_best_params)


Params: {'colsample_bylevel': 0.4292881023563948, 'colsample_bynode': 0.5291136198234517, 'colsample_bytree': 0.4616580256429863, 'learning_rate': 0.4311710058685491, 'max_depth': 2, 'num_boost_round': 20, 'reg_lambda': 1.9578897201213006}
Mean RMSE: 3.9189794846797272
--------------------
Params: {'colsample_bylevel': 0.3625523267576155, 'colsample_bynode': 0.6350638004692478, 'colsample_bytree': 0.6781976563006176, 'learning_rate': 0.14135935551752302, 'max_depth': 3, 'num_boost_round': 20, 'reg_lambda': 1.1423254155608376}
Mean RMSE: 3.9568332989050705
--------------------
Params: {'colsample_bylevel': 0.4408267366563594, 'colsample_bynode': 0.6553579829755966, 'colsample_bytree': 0.14262163491873217, 'learning_rate': 0.018255254802399014, 'max_depth': 4, 'num_boost_round': 1, 'reg_lambda': 4.626362841475587}
Mean RMSE: 4.3665597452036415
--------------------
Params: {'colsample_bylevel': 0.5668940505699103, 'colsample_bynode': 0.6220072889480914, 'colsample_bytree': 0.6871710053396

In [82]:
xgb_stes_spy_rcv_tuned = XGBoostSTESModel(**(DEFAULT_XGBOOST_PARAMS | rcv_best_params))
evaluate_model(spy_data_provider, xgb_stes_spy_rcv_tuned, root_mean_squared_error, 10, 4000)

(4.789720305284427, 5.027242227353158)

### Tuning using Optuna

In [78]:

def xgb_stes_optuna_objective(trial):
    param = {
        "verbosity": 0,
        "num_boost_round": trial.suggest_int("num_boost_round", 1, 20),
        'max_depth': trial.suggest_int("max_depth", 1, 5),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-1, 10.0, log=True),
        "learning_rate": trial.suggest_float("learning_rate", 1e-2, 10.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.3, 0.6),
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.3, 0.6),
        "colsample_bynode": trial.suggest_float("colsample_bylevel", 0.3, 0.6),
    }

    model = XGBoostSTESModel(**param)
    
    indices = np.ones((4000, 1))
    
    tscv = TimeSeriesSplit(n_splits=3)

    rmses = []
    for train_idx, valid_idx in tscv.split(
        indices
    ):
        res, _, _, _ = evaluate_model(
            spy_data_provider,
            model, 
            root_mean_squared_error,
            0,
            valid_idx[0], 
            valid_idx[-1],
        )
        rmses.append(-res)
        
    return np.mean(rmses)



In [79]:
import optuna
from optuna.samplers import TPESampler
sampler = TPESampler(seed = 0)
study_model = optuna.create_study(direction = 'maximize', sampler = sampler)
study_model.optimize(xgb_stes_optuna_objective, n_trials = 100) 

[I 2024-08-13 11:00:19,042] A new study created in memory with name: no-name-2f1b4541-6ec4-4497-a61b-ebbb068fd198


[I 2024-08-13 11:00:19,875] Trial 0 finished with value: -3.983943608905251 and parameters: {'num_boost_round': 11, 'max_depth': 4, 'reg_lambda': 1.6051911333587632, 'learning_rate': 0.4311710058685492, 'colsample_bytree': 0.4270964398016714, 'colsample_bylevel': 0.4937682339199968}. Best is trial 0 with value: -3.983943608905251.
[I 2024-08-13 11:00:20,697] Trial 1 finished with value: -3.991089322355759 and parameters: {'num_boost_round': 9, 'max_depth': 5, 'reg_lambda': 8.459126528049374, 'learning_rate': 0.14135935551752302, 'colsample_bytree': 0.5375175114247993, 'colsample_bylevel': 0.45866847592587134}. Best is trial 0 with value: -3.983943608905251.
[I 2024-08-13 11:00:21,524] Trial 2 finished with value: -4.366837760457006 and parameters: {'num_boost_round': 12, 'max_depth': 5, 'reg_lambda': 0.13869861245357323, 'learning_rate': 0.018255254802399014, 'colsample_bytree': 0.3060655192320977, 'colsample_bylevel': 0.5497859536643814}. Best is trial 0 with value: -3.983943608905251

In [80]:
trial = study_model.best_trial
xgb_stes_spy_best_params_optuna = trial.params
xgb_params_opt_tuned = DEFAULT_XGBOOST_PARAMS | xgb_stes_spy_best_params_optuna
xgb_params_opt_tuned
print('Best params from optuna: \n', pd.DataFrame.from_dict(xgb_params_opt_tuned, orient='index', columns=['Value']))

Best params from optuna: 
                        Value
num_boost_round     6.000000
max_depth           1.000000
learning_rate       0.459328
colsample_bytree    0.540937
colsample_bylevel   0.390439
colsample_bynode    0.800000
reg_lambda          8.292549
random_state       42.000000


In [83]:
xgb_stes_spy_model_opt_tuned = XGBoostSTESModel(**xgb_params_opt_tuned)
data_provider = partial(
    equity_data_provider, 
    tickers=tickers, 
    start_date=start_date, 
    end_date=end_date, 
    model_name=ModelName.XGBoost_STES,
)
evaluate_model(data_provider, xgb_stes_spy_model_opt_tuned, root_mean_squared_error, 10, 4000)

(4.865213034069832, 4.882299983746185)